___
___

## Parallel Computing Code
___
___


## 1) Defining Inputs and Directories.

In [181]:
% INITIATING INPUTS! 
inputs.nmin1 = 3; 
inputs.nmin2 = 1; 
inputs.OnlyTree = 1; 
inputs.Tria = 0; 
inputs.Dist = 1; 
inputs.MinCylRad = 0.0025;
inputs.ParentCor = 1; 
inputs.TaperCor = 1; 
inputs.GrowthVolCor = 0; 
inputs.GrowthVolFac = 1.5; 
inputs.filter.k = 10;
inputs.filter.radius = 0.00;
inputs.filter.nsigma = 1.5;
inputs.filter.PatchDiam1 = 0.05;
inputs.filter.BallRad1 = 0.075;
inputs.filter.ncomp = 2;
inputs.filter.EdgeLength = 0.004;
inputs.filter.plot = 1;
inputs.name = 'tree'; 
inputs.tree = 1;
inputs.model = 1;
inputs.savemat = 1;
inputs.savetxt = 1; 
inputs.plot = 0;
inputs.disp = 0; 

% User Defined ------------------------------------------------------------------------------------------------------------------------------------------------------------
% -------------------------------------------------------------------------------------------------------------------------------------------------------------------------

no_of_iterations =  1;   % Number TRIALS OR ITERATIONS!
       train_set = 18;
       test_set  =  9;
 no_of_model_runs = 5;   % Each RUN for each tree will be repeated 5 times and the average DBH will be taken, The reason is that TreemQSM gives different results even for the same inputs.


lasDirectory = '/mnt/storage/home/aaalshar/TreeQSM/src'; % YOU NEED TO CHANGE THIS! Directory where LAS files are located. 
lasFiles = dir(fullfile(lasDirectory, '*.las'));

DBH_filepath = 'TreeQSM/src/insitu_forest_inventory_mod.xlsx'; % YOU NEED TO CHANGE THIS! Directory where DBH observed Data (Excel sheet) is located.

train_range = ['D' num2str(2) ':D' num2str(train_set+1)]; % I am specifying the cells where the DBH data is stored ((from D2 to D21))
test_range  = ['D' num2str(train_set+2) ':D' num2str(train_set+test_set+1)]; % Extract the cells containing test data.


## 2) Saving the Point Clouds of the Training Set to Perform a Parallel Computing (Make_Models_Parallel).

- Note: Letters were chosen instead of numbers (PA, PB,PC,...) because MATLAB sequence after 1 is 11, 12 and this will cause an issue when trying to compute RMSE. 

In [182]:
for tree = 1:train_set
    name = lasFiles(tree).name;
    lasReader = lasFileReader(name);
    ptCloud = readPointCloud(lasReader);
    P = ptCloud.Location - mean(ptCloud.Location);
    var_name = ['P_' char(tree + 64)];
    eval([var_name ' = P;']);
end

disp("-----------------------")
disp("Saving Trees Completed!")
disp("-----------------------")

variable_names = cell(1, train_set);
for tree = 1:train_set
    variable_names{tree} = ['P_' char(tree + 64)];
end
save('trees_train', variable_names{:});

-----------------------
Saving Trees Completed!
-----------------------


## 3) Perform Training with parallel computing (Make_Models_Parallel)


In [184]:

rmse_old=1e6; % Initiating a large value for RMSE just for the sake of comparison and storing the smallest RMSE. 
rmse_for_iteration= zeros(no_of_iterations, 1); % Creating an empty array to store RMSE for each trial (for monitoring purposes).

DBH_obs = xlsread(DBH_filepath, train_range); % Here We're importing the array of the observed DBH.

for iteration = 1:no_of_iterations % START THE TRIALS LOOP.

inputs.PatchDiam1    = 0.05 + (0.10 - 0.05) * rand;  % The 0.05 is the lower limit and the 0.10 is the upper limit. [0.05 - 0.10]
inputs.PatchDiam2Min = 0.02 + (0.06 - 0.02) * rand;  % The 0.02 is the lower limit and the 0.06 is the upper limit. [0.02 - 0.06]
inputs.PatchDiam2Max = 0.03 + (0.15 - 0.03) * rand;  % The 0.03 is the lower limit and the 0.15 is the upper limit. [0.03 - 0.15]
inputs.BallRad1 = inputs.PatchDiam1+0.015;
inputs.BallRad2 = inputs.PatchDiam2Max+0.01;

QSMs = make_models_parallel( 'trees_train' , 'QSMs trees' , no_of_model_runs , inputs );

% Since the output data structure is complicated, I have to use nested loop to get the output DBH, the below nested loop is just averaging the DBH.
DBH_qsm=zeros(train_set,1);
final_params=zeros(4,1);

k=1;
for i = 1 : no_of_model_runs : no_of_model_runs*train_set
DBH_array=[];

c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
    end
    avg_DBH=mean(DBH_array);

DBH_qsm(k)=avg_DBH;
k = k+1;
end

rmse_score = sqrt(mean((DBH_obs - DBH_qsm).^2));
rmse_for_iteration(iteration)=rmse_score; % HERE I am storing the RMSE for each trial over the 20 trees (training set).
if rmse_score < rmse_old
% Store values in a 4x1 vector
final_params = [inputs.PatchDiam1; inputs.PatchDiam2Min; inputs.PatchDiam2Max; rmse_score]; % FINAL RESULTS CONVENTION.
rmse_old = rmse_score;

end % End for the if statement.
disp(['--------Iteration (' num2str(iteration) ') Completed']);
end


Modelling tree 1/18 (P_A):
Modelling tree 2/18 (P_B):
Modelling tree 3/18 (P_C):
Modelling tree 4/18 (P_D):
Modelling tree 5/18 (P_E):
Modelling tree 6/18 (P_F):
Modelling tree 7/18 (P_G):
Modelling tree 8/18 (P_H):
Modelling tree 9/18 (P_I):
Modelling tree 10/18 (P_J):
Modelling tree 11/18 (P_K):
Modelling tree 12/18 (P_L):
Modelling tree 13/18 (P_M):
Modelling tree 14/18 (P_N):
Modelling tree 15/18 (P_O):
Modelling tree 16/18 (P_P):
Modelling tree 17/18 (P_Q):
Modelling tree 18/18 (P_R):
--------Iteration (1) Completed


## 4) Performing testing on the ~30% of the trees! (After Parallel Computing)

In [185]:
for tree = train_set+1:test_set+train_set
    name = lasFiles(tree).name;
    lasReader = lasFileReader(name);
    ptCloud = readPointCloud(lasReader);
    P = ptCloud.Location - mean(ptCloud.Location);
    var_name = ['P_test_' char(tree + 46)];
    eval([var_name ' = P;']);
end

disp("-----------------------")
disp("Saving Trees Completed!")
disp("-----------------------")

variable_names = cell(1, test_set);
for tree = 1:test_set
    variable_names{tree} = ['P_test_' char(tree + 64)];
end
save('trees_test', variable_names{:});

inputs.PatchDiam1    = final_params(1); % NOW 
inputs.PatchDiam2Min = final_params(2); % THEY
inputs.PatchDiam2Max = final_params(3); % ARE FIXED! 
inputs.BallRad1 = inputs.PatchDiam1+0.015;
inputs.BallRad2 = inputs.PatchDiam2Max+0.01;

disp(["Test DBH From Cells:" test_range])
DBH_obs_test = xlsread(DBH_filepath, test_range); % Here We're importing the array of the observed DBH But now for testing.

QSMs_test = make_models_parallel( 'trees_test' , 'QSMs trees test' , no_of_model_runs , inputs );

k=1;
DBH_qsm_test=[];
for i = 1 : no_of_model_runs : no_of_model_runs*test_set
DBH_array=[];
    c=1;
    for j = i:i+no_of_model_runs-1
        DBH = double(QSMs(j).treedata.DBHqsm);
        DBH_array(c)= DBH;
    c=c+1;
    end
    avg_DBH=mean(DBH_array);

DBH_qsm_test(k)=avg_DBH;
k = k+1;
end

final_test_result(:,1)= 100*DBH_qsm_test;
final_test_result(:,2)= DBH_obs_test;
disp("Final Testing Results:")
disp("   -----------------")
disp("   DBH_qsm   DBH_obs")
disp("   -------   -------")
disp(final_test_result)

-----------------------
Saving Trees Completed!
-----------------------
    "Test DBH From Cells:"    "D20:D28"

Modelling tree 1/9 (P_test_A):
Modelling tree 2/9 (P_test_B):
Modelling tree 3/9 (P_test_C):
Modelling tree 4/9 (P_test_D):
Modelling tree 5/9 (P_test_E):
Modelling tree 6/9 (P_test_F):
Modelling tree 7/9 (P_test_G):
Modelling tree 8/9 (P_test_H):
Modelling tree 9/9 (P_test_I):
Final Testing Results:
   -----------------
   DBH_qsm   DBH_obs
   -------   -------
   16.2384   32.1000
   20.4956   12.0000
   25.9059   19.0000
   18.0383   27.8000
   17.1906   20.9000
   11.7543    6.9000
   14.9588    5.0000
   19.1035   24.0000
   11.3234   38.2000



___
___

## Appendix: Test on synthetic PCD.
- In the below code , I created a PCD for a perfect cylinder to check the functionality of the TreeQSM. The cylinder is 0.3m in diameter and 10m high with 550,000 points. 

In [173]:
% TEST ON A CYLINDER! 

las_file= '/mnt/storage/home/aaalshar/TreeQSM/src/least_squares_fitting/cylinder_file.las';
lasReader = lasFileReader(las_file);
ptCloud = readPointCloud(lasReader);
Pa = ptCloud.Location;
Pa = Pa-mean(Pa);

inputs.PatchDiam1    = 0.1;  
inputs.PatchDiam2Min = 0.2; 
inputs.PatchDiam2Max = 0.4;  
inputs.BallRad1 = inputs.PatchDiam1+0.015;
inputs.BallRad2 = inputs.PatchDiam2Max+0.01;

QSM = treeqsm(P, inputs);

disp("Starting QSM for Synthetic PCD")
disp(['DBH for Synthetic PCD is: ', num2str(100*double(QSM.treedata.DBHqsm)), ' cm'])
disp(["Actual DBH is 40 cm"])

Starting QSM for Synthetic PCD
DBH for Synthetic PCD is: 35.9285 cm
Actual DBH is 40 cm
